In [3]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import requests 
import json
import re

In [4]:
df= pd.read_csv('다이닝_가영_fn.csv')
df

,서칭,이름,주소,위도,경도,전화번호,메뉴/가격,가격대,영업시간,정보,특징,리뷰,평점
0,제육볶음 맛집,무월식탁,서울특별시 강남구 역삼동 812-15,37.502643,127.027311,02-552-9280,"한방바베큐보쌈11,000원간장새우덮밥12,000원제주흑돼지제육쌈밥11,000원",11000원대,영업시간월-금오전 11시 - 오후 10시 (/라스트오더 21:00)토/일오전 11시...,NaN,NaN,37건의 방문자 평가3.6점맛4.2 가격3.2 서비스3.5방문목적점심식사(24) 저...,3.6점
1,제육볶음 맛집,할머니의 레시피,서울특별시 성동구 성수동1가 668-7,37.547597,127.043429,02-467-5101,"고추장불고기(평일점심특선)8,000원버섯불고기(평일점심특선)8,000원쌈밥정식13,...",8000원대,"영업시간월,수,목,금,토/일오전 11시 30분 - 오후 9시 (오찬 11:30~15...",NaN,NaN,21건의 방문자 평가4.2점맛4.7 가격4.3 서비스4.1방문목적점심식사(15) 데...,4.2점
2,제육볶음 맛집,광주식당,서울특별시 동대문구 청량리동 773,37.580576,127.043640,02-969-4403,"청국장6,000원된장찌개6,000원김치찌개7,000원",6000원대,"영업시간화,수,목,금,토/일오전 7시 - 오후 10시휴무일매주 월요일 (매주)",NaN,NaN,18건의 방문자 평가4.2점맛4.8 가격4.9 서비스3.5방문목적점심식사(14) 저...,4.2점
3,제육볶음 맛집,굴다리식당 본관,서울특별시 마포구 도화동 181-45,37.542108,126.950807,02-712-0066,"김치찌개8,000원제육볶음11,000원계란말이10,000원",8000원대,영업시간매일오전 9시 - 오후 10시 (명절 휴무),NaN,NaN,39건의 방문자 평가3.3점맛3.7 가격3.3 서비스3.0방문목적점심식사(21) 저...,3.3점
4,제육볶음 맛집,광화문집,서울특별시 종로구 당주동 43,37.571371,126.974987,02-739-7737,"돼지김치찌개7,000원제육볶음10,000원계란말이5,000원",5000원대,"영업시간매일오전 9시 - 오후 10시 (명절 당일, 다음날 휴무)",NaN,NaN,25건의 방문자 평가3.5점맛4.1 가격4.3 서비스2.8방문목적점심식사(11) 저...,3.5점
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956,완탕면 맛집,NaN,서울특별시 광진구 자양동 606-45,37.531595,127.079207,010-3583-8412,"쇼유완탕7,000원시오완탕7,000원아카완탕8,000원",7000원대,"영업시간매일오전 11시 30분 - 오후 8시 30분 (화 휴무 / 마지막 화,수 휴...",NaN,NaN,맛집 평가하고 300원까지 적립받기,NaN
2957,완탕면 맛집,페이웨이,경기도 하남시 신장동 616,37.545552,127.223680,031-8072-1314,"로메인9,900원",9900원대,영업시간매일오전 10시 30분 - 오전 9시 (매월 1회 백화점 휴무),NaN,NaN,1건의 방문자 평가1.0점맛1.0 가격3.0 서비스1.0방문목적혼밥(1)분위기조용한...,1.0점
2958,완탕면 맛집,청키면가 하남점,경기도 하남시 신장동 616 스타필드하남 3층,37.545552,127.223680,031-8072-8446,"새우완탕면9,500원수교면9,500원완탕과수교면10,500원",9500원대,영업시간매일오전 10시 - 오후 10시,NaN,NaN,5건의 방문자 평가2.7점맛2.9 가격2.9 서비스3.4방문목적점심식사(4) 데이트...,2.7점
2959,완탕면 맛집,누들톡,서울특별시 강남구 역삼동 800 지하,37.506663,127.032005,02-567-1654,"완탕면8,000원탄탄면8,500원히야시소바7,500원",7500원대,영업시간월-금오전 6시 30분 - 오후 10시 30분토요일오전 7시 - 오후 10시더보기,NaN,NaN,블로그리뷰[역삼] 누들톡 - 역삼 하이츠 빌딩 / 면요리 전문점역삼역 근처에서 저녁...,2.5점


In [7]:
a = df[['리뷰']]
a.head(20)

,리뷰
0,37건의 방문자 평가3.6점맛4.2 가격3.2 서비스3.5방문목적점심식사(24) 저...
1,21건의 방문자 평가4.2점맛4.7 가격4.3 서비스4.1방문목적점심식사(15) 데...
2,18건의 방문자 평가4.2점맛4.8 가격4.9 서비스3.5방문목적점심식사(14) 저...
3,39건의 방문자 평가3.3점맛3.7 가격3.3 서비스3.0방문목적점심식사(21) 저...
4,25건의 방문자 평가3.5점맛4.1 가격4.3 서비스2.8방문목적점심식사(11) 저...
5,11건의 방문자 평가3.8점맛4.2 가격3.3 서비스3.7방문목적점심식사(7) 저녁...
6,19건의 방문자 평가4.0점맛4.1 가격4.4 서비스4.2방문목적점심식사(15) 저...
7,12건의 방문자 평가3.9점맛4.4 가격4.4 서비스4.0방문목적점심식사(11) 저...
8,22건의 방문자 평가3.9점맛4.6 가격3.1 서비스3.7방문목적저녁식사(10) 점...
9,15건의 방문자 평가3.8점맛4.6 가격4.4 서비스4.0방문목적저녁식사(9) 점심...


In [9]:
b = df['리뷰'].str.replace('[\{\}\[\]\/?.,;|\)*~`!^\-_+<>@\#$%&\\\=\【】(\'\"\♥\♡\ㅋ\ㅠ\ㅜ\ㄱ\ㅎ\ㄲ\ㅡ]','')
b

0       37건의 방문자 평가36점맛42 가격32 서비스35방문목적점심식사24 저녁식사23 ...
1       21건의 방문자 평가42점맛47 가격43 서비스41방문목적점심식사15 데이트12 식...
2       18건의 방문자 평가42점맛48 가격49 서비스35방문목적점심식사14 저녁식사8 식...
3       39건의 방문자 평가33점맛37 가격33 서비스30방문목적점심식사21 저녁식사17 ...
4       25건의 방문자 평가35점맛41 가격43 서비스28방문목적점심식사11 저녁식사8 술...
                              ...                        
2956                                  맛집 평가하고 300원까지 적립받기
2957    1건의 방문자 평가10점맛10 가격30 서비스10방문목적혼밥1분위기조용한1편의시설무...
2958    5건의 방문자 평가27점맛29 가격29 서비스34방문목적점심식사4 데이트1분위기이국...
2959    블로그리뷰역삼 누들톡  역삼 하이츠 빌딩  면요리 전문점역삼역 근처에서 저녁 먹는다...
2960    5건의 방문자 평가23점맛28 가격10 서비스17방문목적점심식사3 데이트2 여자끼리...
Name: 리뷰, Length: 2961, dtype: object

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."

stop_words = "아무거나, 아무렇게나, 어찌하든지"

stop_words = stop_words.split(' ')

word_tokens = word_tokenize(example)

result = []
for w in word_tokens:
    if w not in stop_words:
        result.append(w)




LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\ILIFO-001/nltk_data'
    - 'C:\\Users\\ILIFO-001\\Anaconda3\\nltk_data'
    - 'C:\\Users\\ILIFO-001\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\ILIFO-001\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ILIFO-001\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [8]:
from konlpy.tag import Hannanum

h = Hannanum()
print(h.pos(u'아니 뭐했다고 시간이 이렇게 빨리가는거지!'))

[('아니', 'I'), ('뭐하', 'P'), ('었다', 'E'), ('고', 'J'), ('시간', 'N'), ('이', 'J'), ('이렇', 'P'), ('게', 'E'), ('빨리가는거', 'N'), ('이', 'J'), ('지', 'E'), ('!', 'S')]
